In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np
from numpy.random import default_rng

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import extract_species_pair_groups

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.linear_model import Ridge

# Utilities
import h5py
import json
import itertools
from copy import deepcopy
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json, save_json

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

# Load train and test splits

In [7]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [8]:
# Load train set and CV indices for Deem
deem_train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/train.idxs', dtype=int)
deem_cv_idxs = np.loadtxt('../Processed_Data/DEEM_330k/cv_2.idxs', dtype=int)

In [ ]:
# Load train set and CV indices for IZA
iza_train_idxs = np.loadtxt('../Processed_Data/IZA_226/train.idxs', dtype=int)
iza_cv_idxs = np.loadtxt('../Processed_Data/IZA_226/cv_2.idxs', dtype=int)

In [11]:
# Load IZA cantons
cantons_iza = np.loadtxt('../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)

In [13]:
# Build set of "master" canton labels
cantons = {}

cantons[4] = np.concatenate((
    cantons_iza[iza_train_idxs], 
    np.ones(len(deem_train_idxs), dtype=int) * 4
))

cantons[2] = np.concatenate((
    np.ones(len(iza_train_idxs), dtype=int),
    np.ones(len(deem_train_idxs), dtype=int) * 2
))

In [ ]:
# Load dummy Deem cantons to test the "null" case
dummy_cantons = {}
dummy_cantons[2] = np.loadtxt('../Processed_Data/DEEM_330k/dummy_cantons_2-class.dat', dtype=int)
dummy_cantons[4] = np.loadtxt('../Processed_Data/DEEM_330k/dummy_cantons_4-class.dat', dtype=int)

In [ ]:
# Concatenate IZA and Deem CV indices
cv_idxs = np.vstack((iza_cv_idxs, deem_cv_idxs + len(iza_train_idxs)))

# Model setup

In [22]:
model_dir = '../Processed_Data/Models'

deem_name = 'DEEM_330k'
iza_name = 'IZA_226'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

In [23]:
# TODO: parameter optimization handling with defaults dictated here

In [24]:
# Global model parameters
# TODO: or use .get_params()?
# TODO: should we make sure break_ties=True?
# TODO: swap dual in linear if features > samples?
svc_kwargs = dict(
    penalty='l2',
    loss='squared_hinge',
    dual=False,
    multi_class='ovr',
    class_weight='balanced',
    fit_intercept=True,
    intercept_scaling=1.0,
    tol=1.0E-3,
    max_iter=1000,
    C=1.0
)

C = np.logspace(-5, 5, 11)
alphas = np.linspace(0.0, 1.0, 11)
regularizations = np.logspace(-12, -1, 2)

# Linear SVC

In [26]:
# Linear model setup
n_species = 2
group_names = {'power': ['OO', 'OSi', 'SiSi', 
                         'OO+OSi', 'OO+SiSi', 'OSi+SiSi',
                         'OO+OSi+SiSi'], 
               'radial': ['O', 'Si', 'O+Si']}

In [27]:
deem_name = 'DEEM_330k'
iza_name = 'IZA_226'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

## Optimize LinearSVC parameters

In [ ]:
# TODO: new function for loading SOAPs

In [30]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    work_dir = f'{model_dir}/{cutoff}/Linear_Models/SVC'
    if cutoff == 3.5: #####
        continue #####
    
    if not os.path.exists(work_dir):
        os.makedirs(work_dir)
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        if spectrum_type == 'radial': ######
            continue #####
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
                
        # TODO: new SOAP loading with concatenation
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            if species_pairing != 'OO+OSi+SiSi': #####
                continue #####
                
            # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
            # TODO: SOAP feature selection (before scaling!)
            # TODO: new SOAP preprocessing with pipeline

            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                if n_cantons == 4: #####
                    continue #####

                # Prepare outputs
                output_dir = f'{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                if not os.path.exists(f'{work_dir}/{output_dir}'):
                    os.makedirs(f'{work_dir}/{output_dir}')
                
                svc_parameter_file = f'{work_dir}/{output_dir}/svc_parameters.json'
                svc_parameters = svc_kwargs['linear'].copy()
                
                # TODO: gridsearchcv with pipeline
                # TODO: save CV run in JSON

0.01



## LR check

In [ ]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    work_dir = f'{model_dir}/{cutoff}/Linear_Models/SVC'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'

        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            x_train = soaps_train[idxs_train_all, :][:, feature_idxs]
            x_test = soaps_test[idxs_test_all, :][:, feature_idxs]

            # Preprocess the SOAPs like the decision functions
            # (i.e., center and scale) for the regression.
            x_train, x_test, x_center, x_scale = \
                utils.preprocess_data(x_train, x_test)
                        
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
                # Load decision functions
                input_dir = f'{n_cantons}-Class/{spectrum_name}/{species_pairing}'

                # Files to store IZA and DEEM KSVC decision functions
                svc_df_train_file = f'{work_dir}/{input_dir}/svc_structure_dfs_train_optimization.dat'
                svc_df_test_file = f'{work_dir}/{input_dir}/svc_structure_dfs_test_optimization.dat'

                df_train = np.loadtxt(svc_df_train_file)[idxs_train_all]
                df_test = np.loadtxt(svc_df_test_file)[idxs_test_all]

                # Center and scale the decision functions
                df_train, df_test, df_center, df_scale = \
                    utils.preprocess_data(df_train, df_test)

                # Check that LR can reproduce the decision functions
                dfp_train, dfp_test = \
                utils.regression_check(x_train, x_test, df_train, df_test, 
                                       regression_type='linear')
            
                print(MAE(dfp_train, df_train))
                print(MAE(dfp_test, df_test))

In [ ]:
# TODO: optimize dummy model